## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-25-14-49-55 +0000,bbc,Farage urges Bank of England boss to stop bond...,https://www.bbc.com/news/articles/c3vz1q5k62eo...
1,2025-09-25-14-48-44 +0000,nyt,Here’s what to know.,https://www.nytimes.com/live/2025/09/25/world/...
2,2025-09-25-14-46-58 +0000,nyt,Trump Assures Arab Leaders He Won’t Allow Isra...,https://www.nytimes.com/2025/09/25/world/middl...
3,2025-09-25-14-45-00 +0000,wsj,Sarkozy Gets Five-Year Prison Sentence in Gadh...,https://www.wsj.com/world/europe/french-court-...
4,2025-09-25-14-42-33 +0000,nyt,What We Know About the Dallas ICE Facility Sho...,https://www.nytimes.com/2025/09/25/us/dallas-i...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2294/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
8,trump,46
28,ice,14
296,president,14
203,new,14
30,shooting,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
26,2025-09-25-14-00-00 +0000,wsj,The Trump administration will offer AI models ...,https://www.wsj.com/tech/ai/trump-elon-musk-xa...,113
244,2025-09-24-21-39-41 +0000,wapo,President Donald Trump wrote on TruthSocial th...,https://www.washingtonpost.com,90
2,2025-09-25-14-46-58 +0000,nyt,Trump Assures Arab Leaders He Won’t Allow Isra...,https://www.nytimes.com/2025/09/25/world/middl...,89
216,2025-09-24-22-55-47 +0000,nypost,Trump’s pro-Ukraine shift a ‘strategic move’ b...,https://nypost.com/2025/09/24/world-news/trump...,86
68,2025-09-25-11-54-03 +0000,nypost,Colin Firth’s ex-wife Livia Giuggioli tears up...,https://nypost.com/2025/09/25/us-news/livia-gi...,83


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
26,113,2025-09-25-14-00-00 +0000,wsj,The Trump administration will offer AI models ...,https://www.wsj.com/tech/ai/trump-elon-musk-xa...
230,73,2025-09-24-22-20-12 +0000,nypost,"Vance tears into Democrats, calls out Gavin Ne...",https://nypost.com/2025/09/24/us-news/vance-te...
39,51,2025-09-25-13-09-54 +0000,bbc,Former French president Sarkozy given five-yea...,https://www.bbc.com/news/articles/cp98kepmj9lo...
169,48,2025-09-25-02-51-33 +0000,nypost,"Mom who lost daughter, 15, to suicide after on...",https://nypost.com/2025/09/24/world-news/mom-w...
2,43,2025-09-25-14-46-58 +0000,nyt,Trump Assures Arab Leaders He Won’t Allow Isra...,https://www.nytimes.com/2025/09/25/world/middl...
92,42,2025-09-25-10-31-18 +0000,nyt,Jimmy Kimmel’s Return Draws 6.2 Million Viewer...,https://www.nytimes.com/2025/09/24/business/me...
99,41,2025-09-25-10-00-00 +0000,nypost,Alleged Charlie Kirk shooter’s trans lover was...,https://nypost.com/2025/09/25/us-news/alleged-...
70,37,2025-09-25-11-47-00 +0000,wsj,The Office of Management and Budget told feder...,https://www.wsj.com/politics/policy/white-hous...
314,35,2025-09-24-17-33-28 +0000,nypost,Zohran Mamdani’s dominant lead in latest NYC m...,https://nypost.com/2025/09/24/us-news/zohran-m...
68,33,2025-09-25-11-54-03 +0000,nypost,Colin Firth’s ex-wife Livia Giuggioli tears up...,https://nypost.com/2025/09/25/us-news/livia-gi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
